# 02: Model

In [1]:
%load_ext autoreload

%autoreload 2

Import libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow_privacy.privacy.optimizers import dp_optimizer_vectorized
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise

import wandb
from wandb.keras import WandbCallback


from synthesizers.cgan.model import (
    ConditionalGAN, 
    GANMonitor
)
from synthesizers.preprocessing.wesad import (
    WESADDataset, 
    LabelType
)
from synthesizers.utils.training import data_split



Set parameters for training

In [3]:
SAMPLING_RATE = 1
USE_SLIDING_WINDOWS = True

# Training Hyperparameters
DP_TRAINING = True
NUM_FEATURES = 6
SEQ_LENGTH = 60
LATENT_DIM = SEQ_LENGTH
BATCH_SIZE = 8
HIDDEN_UNITS = 64
EPOCHS = 420
ACTIVATION = "relu"
RANDOM_SEED = 42
LEARNING_RATE = 0.0002
LOSS_FN = "binary_cross_entropy"
D_ARCHITECTURE = "lstm"
LOSO_TRAINING_WITHOUT_SUBJECT = False # "14"

# DP Training Hyperparameter
L2_NORM_CLIP = 1.0
NUM_MICROBATCHES = BATCH_SIZE
DP_LEARNING_RATE = 1e-3
DELTA = 1e-4


# Define run config
config = {
    "activation_function": ACTIVATION,
    "hidden_units": HIDDEN_UNITS,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "random_seed": RANDOM_SEED,
    "num_features": NUM_FEATURES,
    "seq_length": SEQ_LENGTH,
    "dp_training": DP_TRAINING,
    "learning_rate": LEARNING_RATE,
    "loss_function": LOSS_FN,
    "d_architecture": D_ARCHITECTURE,
    "use_sliding_windows": USE_SLIDING_WINDOWS
}

if LOSO_TRAINING_WITHOUT_SUBJECT:
    config["WESAD_WITHOUT_SUBJ"] = LOSO_TRAINING_WITHOUT_SUBJECT

if DP_TRAINING:
    config["l2_norm_clip"] = L2_NORM_CLIP
    config["num_microbatches"] = NUM_MICROBATCHES
    config["dp_learning_rate"] = DP_LEARNING_RATE


Load data and split it into stress and non-stress for later testing

In [4]:
windows = np.load('data/wesad/wesad_windows.npy')
labels = np.load('data/wesad/wesad_labels.npy')

if USE_SLIDING_WINDOWS:
    mos = windows[labels == 1]
    non_mos = windows[labels == 0]
else:
    mos = windows[labels == 1]
    non_mos = windows[labels == 0]

windows = np.delete(windows, 6, axis=2)
mos = np.delete(mos, 6, axis=2)
non_mos = np.delete(non_mos, 6, axis=2)

num_split = 0.8
trainmos, testmos = data_split(mos, num_split)
trainnomos, testnomos = data_split(non_mos, num_split)

print(trainmos.shape)
print(testmos.shape)
print(trainnomos.shape)
print(testnomos.shape)

(256, 60, 6)
(64, 60, 6)
(610, 60, 6)
(153, 60, 6)


Calculate the noise for privacy guarantees

In [5]:
# get needed noise for target epsilon
min_noise = 1e-5
target_epsilons = [0.1, 1, 10]
noise_multipliers = {target_epsilon : compute_noise(
    windows.shape[0] // 2,
    BATCH_SIZE,
    target_epsilon,
    EPOCHS * 2,
    DELTA,
    min_noise
) for target_epsilon in target_epsilons}
print(noise_multipliers)

compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=windows.shape[0] // 2,
                                              batch_size=BATCH_SIZE,
                                              noise_multiplier=noise_multipliers[target_epsilons[0]],
                                              epochs=EPOCHS*2,
                                              delta=DELTA)

DP-SGD with sampling rate = 1.48% and noise_multiplier = 103.99275642585638 iterated over 56805 steps satisfies differential privacy with eps = 0.1 and delta = 0.0001.
DP-SGD with sampling rate = 1.48% and noise_multiplier = 12.393045830417874 iterated over 56805 steps satisfies differential privacy with eps = 1 and delta = 0.0001.
DP-SGD with sampling rate = 1.48% and noise_multiplier = 1.8509817962014676 iterated over 56805 steps satisfies differential privacy with eps = 10 and delta = 0.0001.
{0.1: 103.99275642585638, 1: 12.393045830417874, 10: 1.8509817962014676}
DP-SGD with sampling rate = 1.48% and noise_multiplier = 103.99275642585638 iterated over 56805 steps satisfies differential privacy with eps = 0.1 and delta = 0.0001.
The optimal RDP order is 128.0.


(0.09999999999999426, 128.0)

Sweep Config only for sweep case

Create TF dataset from windows and labels

In [6]:
# Load dataset into tf dataset
dataset = tf.data.Dataset.from_tensor_slices((windows, labels))

# Shuffle, cache, and batch the dataset
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.cache()
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


tf.random.set_seed(RANDOM_SEED)
randomTrainMos = tf.random.normal(shape=(trainmos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTrainNoMos = tf.random.normal(shape=(trainnomos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTestMos = tf.random.normal(shape=(testmos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTestNoMos = tf.random.normal(shape=(testnomos.shape[0], LATENT_DIM))

Run Training in wandb environment

In [7]:
def main():

    cond_gan = ConditionalGAN(
        num_features=NUM_FEATURES,
        seq_length=SEQ_LENGTH,
        latent_dim=LATENT_DIM,
        discriminator=ConditionalGAN.conditional_discriminator(
            hidden_units=SEQ_LENGTH, 
            seq_length=SEQ_LENGTH, 
            num_features=NUM_FEATURES,
            filters=[32, 64, 32],
            activation_function= ACTIVATION,
            architecture=D_ARCHITECTURE, 
            #head_size=wandb.config.head_size#wandb.config.d_architecture
            #filters=[wandb.config.filter1, wandb.config.filter2, wandb.config.filter3],
            #kernel_sizes=[wandb.config.kernel_size1, wandb.config.kernel_size2, wandb.config.kernel_size3]
            ),
        generator=ConditionalGAN.conditional_generator(
            hidden_units=SEQ_LENGTH, 
            seq_length=SEQ_LENGTH, 
            latent_dim=LATENT_DIM,
            num_features=NUM_FEATURES,
            activation_function=ACTIVATION
        )
    )
    if DP_TRAINING:

        mult = 1 # TODO
        config["noise_multiplier"] = noise_multipliers[mult]

        d_optimizer = dp_optimizer_vectorized.VectorizedDPAdamOptimizer( #vectorized adam am schnellsten
            l2_norm_clip=L2_NORM_CLIP,
            noise_multiplier=noise_multipliers[mult],
            num_microbatches=NUM_MICROBATCHES,
            learning_rate=DP_LEARNING_RATE
        )
    else:
        d_optimizer = Adam(learning_rate=LEARNING_RATE, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    g_optimizer = Adam(learning_rate=LEARNING_RATE, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    cond_gan.compile(
        d_optimizer= d_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5),
        g_optimizer= g_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
        loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    )

    print(f"{cond_gan.d_optimizer} is used")

    history = cond_gan.fit(
        dataset,
        epochs=EPOCHS,
        callbacks=[
            # GANMonitor(
            #     trainmos,
            #     trainnomos,
            #     testmos,
            #     testnomos,
            #     randomTrainMos,
            #     randomTrainNoMos,
            #     randomTestMos,
            #     randomTestNoMos,
            #     num_seq=50,
            #     save_path=None,
            #     batch_size=BATCH_SIZE,
            #     seq_length=SEQ_LENGTH,
            #     num_features=NUM_FEATURES,
            #     dp=DP_TRAINING,
            # )
        ],
    )

    if DP_TRAINING:
        base_path = f"models/new_cgan/{mult}/"
        cond_gan.generator.save(f"{base_path}generator")
        cond_gan.discriminator.save(f"{base_path}discriminator")
    elif LOSO_TRAINING_WITHOUT_SUBJECT:
        base_path = f"models/no_dp/loso/sub{LOSO_TRAINING_WITHOUT_SUBJECT}/{wandb.run.name}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
    else:
        base_path = f"models/no_dp/{wandb.run.name}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
        
main()

discriminator_architecture:  lstm
<tensorflow_privacy.privacy.optimizers.dp_optimizer_vectorized.make_vectorized_optimizer_class.<locals>.DPOptimizerClass object at 0x17ef85580> is used
Epoch 1/420


2023-12-15 19:01:50.131795: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


136/136 [==============================] - 15s 60ms/step - g_loss: 0.6568 - d_loss: 0.5912 - div_term: 0.5782
Epoch 2/420
136/136 [==============================] - 8s 62ms/step - g_loss: 2.5667 - d_loss: 0.0515 - div_term: 0.1131
Epoch 3/420
136/136 [==============================] - 9s 63ms/step - g_loss: 5.0414 - d_loss: 0.0046 - div_term: 0.0263
Epoch 4/420
136/136 [==============================] - 9s 63ms/step - g_loss: 5.8839 - d_loss: 0.0021 - div_term: 0.0321
Epoch 5/420
136/136 [==============================] - 9s 63ms/step - g_loss: 6.2546 - d_loss: 0.0578 - div_term: 0.1628
Epoch 6/420
136/136 [==============================] - 9s 65ms/step - g_loss: -0.0814 - d_loss: 0.6563 - div_term: 0.8287
Epoch 7/420
136/136 [==============================] - 9s 63ms/step - g_loss: 0.2609 - d_loss: 0.6369 - div_term: 0.6330
Epoch 8/420
136/136 [==============================] - 9s 63ms/step - g_loss: -0.3528 - d_loss: 0.7045 - div_term: 0.9023
Epoch 9/420
136/136 [====================

INFO:tensorflow:Assets written to: models/new_cgan/1/generator/assets


INFO:tensorflow:Assets written to: models/new_cgan/1/generator/assets


INFO:tensorflow:Assets written to: models/new_cgan/1/discriminator/assets


INFO:tensorflow:Assets written to: models/new_cgan/1/discriminator/assets


In [8]:
import os
os.system("say 'C GAN IST FERTIG'")

0